In [9]:
import os
import re

In [11]:
def list_wiki(dirname) :
    filepaths = []
    filenames = os.listdir(dirname)
    
    for filename in filenames :
        filepath = os.path.join(dirname, filename)
        
        if os.path.isdir(filepath) :
            filepaths.extend(list_wiki(filepath))
        else :
            find = re.findall(r"wiki_[0-9][0-9]",filepath)
            if 0 < len(find) :
                filepaths.append(filepath)
                
    return sorted(filepaths)

In [12]:
filepaths = list_wiki("F:/Study/DataSet/text")

In [15]:
print(filepaths[:5])
print(len(filepaths))

['F:/Study/DataSet/text\\AA\\wiki_00', 'F:/Study/DataSet/text\\AA\\wiki_01', 'F:/Study/DataSet/text\\AA\\wiki_02', 'F:/Study/DataSet/text\\AA\\wiki_03', 'F:/Study/DataSet/text\\AA\\wiki_04']
676
